In [ ]:
#delete the empty folder
import os
import shutil
data_dir = '/home/yuhaowang/data/processed_data'
for dir in os.listdir(data_dir):
    data_path = os.path.join(data_dir, dir,'output')
    for slide in os.listdir(data_path):
        slide_dir =os.path.join(data_path,slide)
        #print(slide_dir)
        #if slide_dir is not dir,remove it
        if not os.path.isdir(slide_dir):
            #print(slide_dir)
            os.remove(slide_dir)
            continue
        if len(os.listdir(slide_dir)) <3:
            print(slide_dir)
            #remove this empty dir
            #remove even it not empty
            shutil.rmtree(slide_dir)
            #break
        

In [ ]:
import multiprocessing as mp
from pathlib import Path
import h5py
import os

def read_assets_from_h5(h5_path: str):
    """Read the assets from the h5 file"""
    assets = {}
    attrs = {}
    try:
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
    except Exception as e:
        print(f'Error reading {h5_path}: {e}')
        return None, None

def process_file(file_path):
    try:
        data, _ = read_assets_from_h5(file_path)
        if data is None:
            print('Removing corrupt file:', file_path)
            os.remove(file_path)
            return

        feature = data.get('features')
        coords = data.get('coords')
        
        if coords is None or feature is None:
            print('Removing file with missing data:', file_path)
            os.remove(file_path)
            return

        if len(coords.shape) != 2:
            print('Removing file with invalid coords shape:', file_path)
            os.remove(file_path)
            return

        if feature.shape[0] <= 80:
            print('Removing file due to low dimension:', file_path)
            os.remove(file_path)
    except Exception as e:
        print(f'Error processing {file_path}: {e}')
        os.remove(file_path)

def main():
    # 获取指定路径下所有以.h5结尾的文件
    h5_file_list = list(Path('/data4/embedding/temp').rglob('*.h5'))
    
    # 打印找到的文件总数
    print(f'Total files found: {len(h5_file_list)}')
    
    # 使用多进程池处理文件
    with mp.Pool(processes=mp.cpu_count()) as pool:
        pool.map(process_file, h5_file_list)
    
if __name__ == "__main__":
    main()

Total files found: 39603
Removing file due to low dimension:Removing file due to low dimension:Removing file due to low dimension:   /data4/embedding/temp/embedding/private_chunk_4/UNI-2/S19071.h5/data4/embedding/temp/embedding/HIST2ST/UNI-2/C3.h5/data4/embedding/temp/embedding/private_chunk_4/UNI-2/S15659.h5


Removing file due to low dimension: /data4/embedding/temp/embedding/HIST2ST/UNI-2/A2.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S19812.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S06225.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_4/UNI-2/S11574.h5
Removing file due to low dimension:Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S11585.h5
 /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S11614.h5Removing file due to low dimension:
 /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S09545.h5

# 下面这个板块要着重处理embedding命令的问题

In [ ]:
#BCNB数据集 出来
import os
import re
#处理BCNB数据集
def rename_h5_files(root_folder):
    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".h5") and re.match(r"^\d", filename):  # 判断是否以数字开头
                old_path = os.path.join(foldername, filename)
                new_filename = f"AIDPATH_{filename}"
                new_path = os.path.join(foldername, new_filename)
                
                if not os.path.exists(new_path):
                    os.rename(old_path, new_path)
                    print(f"Renamed: {old_path} -> {new_path}")
                else:
                    if os.path.exists(new_path):
                        #delete old_path
                        os.remove(old_path)
                    print(f"Skipping {old_path}, target file already exists.")

if __name__ == "__main__":
    root_directory = '/data4/embedding/AIDPATH'
    if os.path.exists(root_directory) and os.path.isdir(root_directory):
        rename_h5_files(root_directory)
    else:
        print("提供的路径无效，请输入正确的文件夹路径。")

Skipping /data4/embedding/AIDPATH/TITAN/2058.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2034.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/1916.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/1991.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2063.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/1818.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2051.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/1882.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2068.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2074.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/1815.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2135.h5, target file already exists.
Skipping /data4/embedding/AIDPATH/TITAN/2053.h5, target file already exists.

In [10]:
import h5py
def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets frdom the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

test_path = '/data4/embedding/private_chunk_1/FMBC/S18407.h5'
assets, attrs = read_assets_from_h5(test_path)
assets['features']

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/data4/embedding/private_chunk_1/FMBC/S18407.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## 删除某些测试的不太行的特征，重新提取


In [31]:
import os
import shutil
embedding_dir ='/data4/embedding'
for dataset in os.listdir(embedding_dir):
    dataset_path = os.path.join(embedding_dir, dataset)
    for model in os.listdir(dataset_path):
        if model == 'FMBC':
            print(os.path.join(dataset_path, model))
            shutil.rmtree(os.path.join(dataset_path, model))

/data4/embedding/TCGA-BRCA/FMBC
/data4/embedding/AIDPATH/FMBC
/data4/embedding/BRACS/FMBC
/data4/embedding/IMPRESS/FMBC
/data4/embedding/BCNB/FMBC


## 重新命名processed data，也就是patch文件夹的命名方式，保证和最后的特征一致


In [ ]:
import os
import re
import os
data_dir = '/data4/processed_data/'
aidpath_dir = os.path.join(data_dir, 'AIDPATH')
for folder in os.listdir(os.path.join(aidpath_dir, 'output')):

    new_folder_name = 'AIDPATH_'+folder
    new_folder_path = os.path.join(aidpath_dir, 'output', new_folder_name)
    #print(new_folder_path)

    os.rename(os.path.join(aidpath_dir, 'output', folder), new_folder_path)

tcga_dir = os.path.join(data_dir, 'TCGA-BRCA','output')
for folder in os.listdir(os.path.join(tcga_dir)):
    match = re.match(r"(TCGA-[A-Z0-9]+-[A-Z0-9]+-[0-9]+)", folder)
    new_name = match.group(1) + ".svs"
    #print(new_name)
    try:
        new_path = os.path.join(tcga_dir, new_name)
        os.rename(os.path.join(tcga_dir, folder), new_path)
    except:
        print(f"Failed to rename {folder} to {new_name}")

import os

bcnb_dir = os.path.join(data_dir, 'BCNB')
for folder in os.listdir(os.path.join(bcnb_dir, 'output')):

    new_folder_name = 'bcnb_'+folder
    new_folder_path = os.path.join(bcnb_dir, 'output', new_folder_name)
    #print(new_folder_path)

    os.rename(os.path.join(bcnb_dir, 'output', folder), new_folder_path)

## 这个文件夹主要移动AHSL的数据集，构建AHSL的FMBC数据

In [4]:

import glob
import os
origin_dir = '/data4/embedding/'
standard_dir = '/data4/embedding/AHSL/CONCH'
#matching h5 files and path contain 'FMBC'
FMBC_file_path_list = glob.glob(os.path.join(origin_dir, '**', '*.h5'), recursive=True)
#filter file with FMBC
FMBC_file_path_list = [file for file in FMBC_file_path_list if 'FMBC' in file]
    

In [6]:
target_dir = '/data4/embedding/AHSL/FMBC'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
import shutil

    #retrival file path in FMBC_file_path_list by file
file_actu_path = [file for file in FMBC_file_path_list if os.path.basename(file) in os.listdir(standard_dir)]

In [11]:
#copy file_actu_path to target_dir
for file in file_actu_path:
    try:
        shutil.copy(file, target_dir)
    except:
        print(file)